In [3]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

# Задание:
- Покадрово считаете видео и копите кадры в вектор (50 кадров например)
- Для каждого пикселя (50 точек) ищем медиану, сохраняем в соответсвующую координату маски
- 200*200 картинка отскейлить нужно, иначе долго считать будет
- Сравнить текущий кадр с моделью фона - если сильно отличается передний план, иначе фон 
- Применить фильтр против шума можно готовую функцию

In [185]:
def is_different(background_frame, median_frame, barrier):
    gray_current_frame = cv2.cvtColor(median_frame, cv2.COLOR_BGR2GRAY)
    gray_background_model = cv2.cvtColor(background_frame, cv2.COLOR_BGR2GRAY)
    
    difference = cv2.absdiff(gray_current_frame, gray_background_model)
    
    _, thresholded = cv2.threshold(difference, 50, 255, cv2.THRESH_BINARY)
    
    foreground_pixels = cv2.countNonZero(thresholded)
    
    if foreground_pixels > barrier * (median_frame.shape[0] * median_frame.shape[1]):
        return True
    else:
        return False
        
def read_stream_from_webcam():
    cap = cv2.VideoCapture(0) 
    
    img_buff = []
    background = None
    get_step = 0

    try:
        while(cap.isOpened()): 
            ret, frame = cap.read() 
            if ret is True:
                
                if get_step < STACK_GAP:
                    width = 200
                    height = 200
                    
                    dim = (width, height)
                    resized = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
                    if get_step == 0:
                        if background is None:
                            print("back configured")
                            background = resized
                    else:
                        img_buff.append(resized)
                    
                    get_step += 1
                    
                else:
                    median = np.median(img_buff, axis=0)
                    clipped_median = np.clip(median, a_min=0, a_max=256)
                    clipped_median = clipped_median.astype(np.uint8)
                    
                    if background is not None and is_different(background, clipped_median, barrier=0.25):
                        clipped_median = cv2.fastNlMeansDenoisingColored(clipped_median, None, 10, 10, 7, 21)
                        image = clipped_median

                        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                        
                        hsv_image[:, :, 0] = hsv_image[:, :, 0] * 10
                        hsv_image[:, :, 1] = hsv_image[:, :, 1] * 2 
                        hsv_image[:, :, 2] = hsv_image[:, :, 2] * 2 
                        
                        red_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

                        image = red_image
                        cv2.imshow('frame', image)
                        
                    else:
                        background = cv2.fastNlMeansDenoisingColored(background, None, 10, 10, 7, 21)
                        image = background
                        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

                        hsv_image[:, :, 0] = hsv_image[:, :, 0] * 1 
                        hsv_image[:, :, 1] = hsv_image[:, :, 1] * 1 
                        hsv_image[:, :, 2] = hsv_image[:, :, 2] * 1 
                        
                        green_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

                        image = green_image
                        cv2.imshow('frame', image) 
                    
                    get_step = 0
                    img_buff = []
        
                if cv2.waitKey(1) & 0xFF == ord('x'): 
                    break
    finally:
        cap.release() 
        cv2.destroyAllWindows()

In [186]:
GET_FRAME_GAP = 1
STACK_GAP = 50

read_stream_from_webcam()

back configured
